In [ ]:
### Load libraries
library(here)
library(rgeos)
library(geosphere)
library(rgdal)
library("ggplot2")
library("tidyr")
library(raster) # for crs
library(ggbeeswarm)
library(tmap)
library(sf)
library(stringr)
library(scales)

In [2]:
### Load utility functions and themes
source(here("R","getNicheOutline.R"))
source(here("R","theme_tmaps.R"))

In [3]:
### Define paths

# Define imputs paths:

# Admnistrative map of China
path2china<-here('raw_data','CHN_adm')

# Archaeological records location data
path2buckwheat<-here("data","ESM_2.csv") # Buckwheat's occurrence records
path2locations<-here("data","site_locations.csv")

# Production and predictions data
path2predictions <- here("outputs","predictions")

#### Paths to outputs:

path2out<-here("outputs")

In [4]:
# Import data:
loc<-read.csv(path2locations)
loc$period<-factor(loc$period,levels=paste(c(seq(1000,14000,1000),17000),"BP"),ordered=TRUE)
coordinates(loc)<-~Longitude+Latitude

In [5]:
# Import the outer border of China
outer_border<-st_read(dsn=path2china,layer='CHN_adm0')
# Import the spatial polygons data frame of counties:
china<-readOGR(dsn=path2china,layer='CHN_adm3')
# Import production data

Reading layer `CHN_adm0' from data source `G:\My Drive\my_repositories\PhD\raw_data\CHN_adm' using driver `ESRI Shapefile'
Simple feature collection with 1 feature and 70 fields
geometry type:  MULTIPOLYGON
dimension:      XY
bbox:           xmin: 73.5577 ymin: 18.15931 xmax: 134.7739 ymax: 53.56086
geographic CRS: WGS 84
OGR data source with driver: ESRI Shapefile 
Source: "G:\My Drive\my_repositories\PhD\raw_data\CHN_adm", layer: "CHN_adm3"
with 2409 features
It has 13 fields
Integer64 fields read as strings:  ID_0 ID_1 ID_2 ID_3 


In [6]:
# List of crops in the analysis:

crops<-c("rice","millet","buckwheat","wheat","barley")

In [7]:
for (crop in crops){
    print(crop)
    path2pred_summary<-paste(path2predictions,"/",crop,"_predictions.csv",sep="")
    pred_summary<-read.csv(path2pred_summary,row.names=1)

    ### Get the log means predictions, 0.05, and 0.95 quantiles for each county in a separate variables
    mean_names<-colnames(pred_summary)[grepl("log",colnames(pred_summary))&grepl("mean",colnames(pred_summary))]
    mean<-pred_summary[,colnames(pred_summary)[grepl("log",colnames(pred_summary))&grepl("mean",colnames(pred_summary))]]
    q5_names<-colnames(pred_summary)[grepl("log",colnames(pred_summary))&grepl("X5",colnames(pred_summary))]
    q5<-pred_summary[,colnames(pred_summary)[grepl("log",colnames(pred_summary))&grepl("X5",colnames(pred_summary))]]
    q95_names<-colnames(pred_summary)[grepl("log",colnames(pred_summary))&grepl("X95",colnames(pred_summary))]
    q95<-pred_summary[,colnames(pred_summary)[grepl("log",colnames(pred_summary))&grepl("X95",colnames(pred_summary))]]

    # Put the values together in the dataframe
    df<-cbind(mean,q5,q95)
    colnames(df)<-paste(crop,colnames(df),sep="_")
    # Simplify colum names a bit
    colnames(df)<-gsub("s_pred","",colnames(df))
    colnames(df)<-gsub("_X","",colnames(df))
    colnames(df)<-gsub("\\.","",colnames(df))
    
    # Bind data to spatial polygons
    china@data<-cbind(china@data,df)
    }

[1] "rice"
[1] "millet"
[1] "buckwheat"
[1] "wheat"
[1] "barley"


In [8]:
### Transform china spatial points dataframe to simple features
china_sf<-st_as_sf(china) 

In [9]:
### Bind the location data, with the predictions for counties in which buckwheat was found:
proj4string(loc) <- crs(china)
pred_data<-sp::over(loc,china)
loc@data<-cbind(loc@data,pred_data)

In [10]:
crops<-c("rice","millet","buckwheat","wheat","barley")

In [ ]:
timeslices <-levels(loc@data$period)

for (crop in (crops)){
    
    columns<-grep(paste("^",crop,sep=""),tolower(colnames(loc@data)))
    if(crop%in%c("millet","buckwheat")){columns<-grep(crop,tolower(colnames(loc@data)))}
    columns<-columns[!(columns%in%grep("_log",tolower(colnames(loc@data))))]
    for (column in columns){
        for (i in timeslices[1:length(timeslices)]){
            print(crop)
            print(column)
            print(i)
            loc@data[loc@data$period==i&!is.na(loc@data[,column]),paste(crop,c('Mean','5','95'),sep="")]<- loc@data[loc@data$period==i&!is.na(loc@data[,column]),colnames(loc@data) %in% paste(crop,c("_mean","5","95"),"_",gsub(" BP","",i),"_log",sep="")]
        }
    }
}

### Get the outlines

In [12]:
path2out<-here("outputs")

In [ ]:
for(crop in crops){
    print(crop)
    # Set up
    t<-loc@data[,c("period",paste(crop,"Mean",sep=""))]
    colnames(t)[2]<-"threshold"
    t<-t[!is.na(t$threshold),]
    minimum<-unique(sort(t$threshold))[1]
    if(crop%in%c("millet","barley")){
        outlier<-minimum
        minimum<- unique(sort(t$threshold))[4]
    }else if(crop=="wheat"){
        outlier<-minimum
        minimum<-unique(sort(t$threshold))[3]}
         
    # Gat maximum extent and outliers:
    ts<-aggregate(threshold~period,data=t,FUN="min")
    ts<-ts[ts$period>t[t$threshold==minimum,"period"] & ts$threshold>minimum,]   

    # This just gets the maximum outline
    facets<-unique(t$period)
    columns<-unique(gsub(" BP","_log",paste(crop,"mean",t$period,sep="_")))
    ord<-as.numeric(gsub("_log","",gsub(paste(crop,"_mean_",sep=""),"",columns)))
    columns<-columns[order(ord)]
    facets<-facets[order(ord)]
    print("maximu")
         
    max_border<-getNicheOutline(columns,threshold=minimum,threshold_max=NULL,facets,sf_object=china_sf)
         
    if(crop%in%c("millet","wheat","barley")){
        print("outliers")
        outlier_border<-getNicheOutline(columns,threshold=outlier,threshold_max=NULL,facets,sf_object=china_sf)
    }
   
    # This gets period specific outlines
    print("period")  
    columns<-gsub(" BP","_log",paste(crop,"mean",ts$period,sep="_"))
    ord<-as.numeric(gsub("_log","",gsub(paste(crop,"_mean_",sep=""),"",columns)))
    columns<-columns[order(ord)]
    facets<-facets[order(ord)]
    thresholds<-ts$threshold
    facets<- ts$period
   
    period_border<-getNicheOutline(columns=columns,thresholds,threshold_max=NULL,facets,sf_object=china_sf)
   
    # Define extent types   
    max_border@data$threshold<-"maximum extent"
    if (crop!="rice"){outlier_border@data$threshold<-"outliers extent"}
    period_border@data$threshold<-"extent by period"
   
    period_border@data$id <- seq(max(max_border@data$id)+1,length.out=nrow(period_border@data))
    if(crop=="millet"){
        outlier_border@data$id <- seq(max(period_border@data$id)+1,length.out=nrow(outlier_border@data))
    }
    if(crop=="rice"){niche <- rbind(max_border,period_border)}else{niche <- rbind(max_border,period_border, outlier_border)}
    niche@data$facet<-as.character(niche@data$facet)
    writeOGR(niche,dsn=paste(path2out,"/",crop,"_niche",sep=""),layer=paste(crop,"niche",sep="_"),driver="ESRI Shapefile")
   }